# ETL: Strombörse
---
Diese Komponente ist für das Extrahieren, Transformieren und Laden von Daten aus der Quelle [SMARD](https://www.smard.de/home/downloadcenter/download-marktdaten/?downloadAttributes=%7B%22selectedCategory%22:3,%22selectedSubCategory%22:8,%22selectedRegion%22:%22DE%22,%22selectedFileType%22:%22XLSX%22,%22from%22:1672527600000,%22to%22:1704063599999%7D) zuständig. Die Komponente verarbeitet eine Quelldatei, die die auf dem Day-Ahead-Markt ([EPEX SPOT](https://www.epexspot.com/en/market-data?market_area=DE-LU&trading_date=2024-03-31&delivery_date=2024-04-01&underlying_year=&modality=Auction&sub_modality=DayAhead&technology=&product=60&data_mode=graph&period=&production_period=)) festgelegten Strompreise darstellt.

# Funktionsweise des Prozesses
---
In diesem Prozess werden die folgenden Funktionen verwendet:



1. `extraction_and_processing_dataset_SMARD()`: Diese Funktion extrahiert aus einer Quelldatei die Marktdaten zu den Großhandelsstrompreisen für das Jahr 2023. Diese Daten stellen die Strompreise dar, die auf dem Day-Ahead-Markt in 15-Minuten-Intervallen festgelegt werden.


2. `run_ETL_Stromboerse()`: Funktion zur Ausführung des Prozesses.



In [16]:
import openpyxl
import os

def extraction_and_processing_dataset_SMARD():
    try:
        # Definition der Pfade für die Eingabe- und Ausgabedateien
        input_folder = "Datenquelle/SMARD-Daten/Viertelstunde"
        input_file = "Gro_handelspreise_202301010000_202312312359_Viertelstunde.xlsx"
        input_path = os.path.join(input_folder, input_file)
        
        output_folder = "Output-Data"
        output_file = "ETL-Prozess_Ergebnisse.xlsx"  # Änderung des Ausgabedateinamens
        output_path = os.path.join(output_folder, output_file)
        
        # Öffnen der Excel-Datei
        workbook = openpyxl.load_workbook(input_path)

        # Öffnen des gewünschten Blatts
        sheet = workbook["Großhandelspreise"]

        # Öffnen der Zieltabelle (Simulationsparameter) im Ergebnis-Excel
        try:
            new_workbook = openpyxl.load_workbook(output_path)
        except FileNotFoundError:
            new_workbook = openpyxl.Workbook()

        new_sheet = new_workbook["Simulationsparameter"] if "Simulationsparameter" in new_workbook.sheetnames else new_workbook.active
        
        # Kopieren der Daten in die Spalte L der Zieltabelle ab der Zeile 1
        end_row = sheet.max_row
        column_to_copy = 4

        for row in range(2, end_row + 1):
            # Kopieren des Werts von Spalte L (12) zur Spalte M (13) und Teilen durch 1000
            value_to_copy = sheet.cell(row=row+9, column=column_to_copy).value
            new_sheet.cell(row=row, column=12).value = value_to_copy
            
            if value_to_copy is not None:
                new_sheet.cell(row=row, column=13).value = value_to_copy / 1000  # Wert in Spalte M (13) teilen durch 1000

        # Setzen des Headers
        new_sheet['L1'] = 'Strommarktpreis [€/MWh]'
        new_sheet['M1'] = 'Strommarktpreis [€/kWh]'
        
        # Speichern der neuen Excel-Datei
        new_workbook.save(output_path)

        print("- ETL-Prozess_Strombörse: Aufgabe 1/1 erledigt")
    except Exception as e:
        print("Ein Fehler ist aufgetreten:", e)


In [17]:
def run_ETL_Stromboerse():
    extraction_and_processing_dataset_SMARD()